## 1. Import  Libraries


In [60]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 36.0 MB/s eta 0:00:00:00:0100:01


In [9]:
!pip install feature_engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.1/357.1 kB 9.2 MB/s eta 0:00:00ta 0:00:01


In [180]:
import os

import pickle

import boto3

import xgboost as xgb

from sklearn.metrics import r2_score

import numpy as np

import pandas as pd

import sklearn

from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestRegressor

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.metrics.pairwise import rbf_kernel

from sklearn.preprocessing import (
	OneHotEncoder,
    OrdinalEncoder,
    MinMaxScaler,
    PowerTransformer,
    StandardScaler,
	FunctionTransformer
)

from feature_engine.outliers import Winsorizer

from sklearn.compose import ColumnTransformer

from feature_engine.datetime import DatetimeFeatures

from feature_engine.selection import SelectBySingleFeaturePerformance

from feature_engine.encoding import (
	RareLabelEncoder,
	MeanEncoder,
	CountFrequencyEncoder
)

import warnings

import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.tuner import (
    IntegerParameter,
    ContinuousParameter,
    HyperparameterTuner
)

## 2. Display Settings

In [102]:
pd.set_option("display.max_columns",None)

sklearn.set_config(transform_output= "pandas")

warnings.filterwarnings("ignore")

## 3. Read the Dataset

In [103]:
train = pd.read_csv("train.csv")
train

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Spicejet,2019-05-15,Kolkata,Banglore,15:55:00,21:25:00,330,1.0,No Info,7804
1,Jet Airways,2019-05-18,Delhi,Cochin,13:25:00,12:35:00,1390,2.0,No Info,15129
2,Indigo,2019-06-06,Delhi,Cochin,23:30:00,04:35:00,305,1.0,No Info,10112
3,Jet Airways,2019-05-01,Kolkata,Banglore,14:05:00,08:15:00,1090,1.0,In-flight meal not included,8586
4,Jet Airways,2019-04-06,Banglore,Delhi,07:10:00,10:10:00,180,0.0,In-flight meal not included,4544
...,...,...,...,...,...,...,...,...,...,...
635,Indigo,2019-03-03,Delhi,Cochin,05:05:00,10:05:00,300,1.0,No Info,9581
636,Indigo,2019-06-27,Kolkata,Banglore,15:15:00,17:45:00,150,0.0,No Info,4804
637,Jet Airways,2019-05-18,Kolkata,Banglore,20:00:00,09:20:00,800,1.0,In-flight meal not included,10844
638,Vistara,2019-04-15,Banglore,Delhi,17:00:00,19:35:00,155,0.0,No Info,5613


In [104]:
val = pd.read_csv("val.csv")
val

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Vistara,2019-03-06,Mumbai,Hyderabad,21:55:00,16:45:00,1130,1.0,No Info,12395
1,Jet Airways,2019-06-12,Delhi,Cochin,09:35:00,12:35:00,1620,2.0,No Info,13292
2,Jet Airways,2019-06-12,Delhi,Cochin,18:25:00,12:35:00,1090,2.0,No Info,14237
3,Air India,2019-05-18,Kolkata,Banglore,08:20:00,21:50:00,810,2.0,No Info,10697
4,Jet Airways,2019-03-12,Banglore,New Delhi,08:55:00,10:25:00,1530,1.0,In-flight meal not included,11087
...,...,...,...,...,...,...,...,...,...,...
155,Jet Airways,2019-05-21,Kolkata,Banglore,21:10:00,08:15:00,665,1.0,In-flight meal not included,10844
156,Jet Airways,2019-06-12,Kolkata,Banglore,20:00:00,12:00:00,960,1.0,In-flight meal not included,8529
157,Jet Airways,2019-05-09,Delhi,Cochin,17:30:00,19:00:00,1530,1.0,No Info,15554
158,Multiple Carriers,2019-05-27,Delhi,Cochin,10:00:00,01:30:00,930,1.0,No Info,9508


In [105]:
test = pd.read_csv("test.csv")
test

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Multiple Carriers,2019-06-03,Delhi,Cochin,08:00:00,19:15:00,675,1.0,No Info,15058
1,Goair,2019-05-01,Delhi,Cochin,07:25:00,13:35:00,370,1.0,No Info,5842
2,Spicejet,2019-04-12,Kolkata,Banglore,06:55:00,09:30:00,155,0.0,No check-in baggage included,3841
3,Multiple Carriers,2019-03-06,Delhi,Cochin,07:00:00,18:50:00,710,1.0,No Info,15867
4,Air India,2019-04-09,Delhi,Cochin,05:55:00,07:40:00,1545,2.0,No Info,10021
...,...,...,...,...,...,...,...,...,...,...
195,Jet Airways,2019-06-09,Kolkata,Banglore,09:35:00,10:55:00,1520,1.0,No Info,14151
196,Jet Airways,2019-06-15,Delhi,Cochin,11:30:00,04:25:00,1015,1.0,In-flight meal not included,10262
197,Indigo,2019-03-01,Kolkata,Banglore,17:15:00,19:55:00,160,0.0,No Info,6565
198,Spicejet,2019-06-24,Kolkata,Banglore,06:55:00,09:30:00,155,0.0,No check-in baggage included,3841


## 4. Preprocessing Operations

In [128]:
# airline
air_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
    ("encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))
])

#doj
feature_to_extract = ["month", "week", "day_of_week", "day_of_year"]

doj_transformer = Pipeline(steps=[
    ("dt", DatetimeFeatures(features_to_extract=feature_to_extract, yearfirst=True, format="mixed")),
    ("scaler", MinMaxScaler())
])

# source & destination
location_pipe1 = Pipeline(steps=[
    ("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
    ("encoder", MeanEncoder()),
    ("scaler", PowerTransformer())
])

def is_north(X):
    columns = X.columns.to_list()
    north_cities = ["Delhi", "Kolkata", "Mumbai", "New Delhi"]
    return (
        X
        .assign(**{
            f"{col}_is_north": X.loc[:, col].isin(north_cities).astype(int)
            for col in columns
        })
        .drop(columns=columns)
    )

location_transformer = FeatureUnion(transformer_list=[
    ("part1", location_pipe1),
    ("part2", FunctionTransformer(func=is_north))
])

# dep_time & arrival_time
time_pipe1 = Pipeline(steps=[
    ("dt", DatetimeFeatures(features_to_extract=["hour", "minute"])),
    ("scaler", MinMaxScaler())
])

def part_of_day(X, morning=4, noon=12, eve=16, night=20):
    columns = X.columns.to_list()
    X_temp = X.assign(**{
        col: pd.to_datetime(X.loc[:, col]).dt.hour
        for col in columns
    })

    return (
        X_temp
        .assign(**{
            f"{col}_part_of_day": np.select(
                [X_temp.loc[:, col].between(morning, noon, inclusive="left"),
                 X_temp.loc[:, col].between(noon, eve, inclusive="left"),
                 X_temp.loc[:, col].between(eve, night, inclusive="left")],
                ["morning", "afternoon", "evening"],
                default="night"
            )
            for col in columns
        })
        .drop(columns=columns)
    )

time_pipe2 = Pipeline(steps=[
    ("part", FunctionTransformer(func=part_of_day)),
    ("encoder", CountFrequencyEncoder()),
    ("scaler", MinMaxScaler())
])

time_transformer = FeatureUnion(transformer_list=[
    ("part1", time_pipe1),
    ("part2", time_pipe2)
])

# duration
class RBFPercentileSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, variables=None, percentiles=[0.25, 0.5, 0.75], gamma=0.1):
        self.variables = variables
        self.percentiles = percentiles
        self.gamma = gamma


    def fit(self, X, y=None):
        if not self.variables:
            self.variables = X.select_dtypes(include="number").columns.to_list()

        self.reference_values_ = {
            col: (
                X
                .loc[:, col]
                .quantile(self.percentiles)
                .values
                .reshape(-1, 1)
            )
            for col in self.variables
        }

        return self


    def transform(self, X):
        objects = []
        for col in self.variables:
            columns = [f"{col}_rbf_{int(percentile * 100)}" for percentile in self.percentiles]
            obj = pd.DataFrame(
                data=rbf_kernel(X.loc[:, [col]], Y=self.reference_values_[col], gamma=self.gamma),
                columns=columns
            )
            objects.append(obj)
        return pd.concat(objects, axis=1)
    

def duration_category(X, short=180, med=400):
    return (
        X
        .assign(duration_cat=np.select([X.duration.lt(short),
                                    X.duration.between(short, med, inclusive="left")],
                                    ["short", "medium"],
                                    default="long"))
        .drop(columns="duration")
    )

def is_over(X, value=1000):
    return (
        X
        .assign(**{
            f"duration_over_{value}": X.duration.ge(value).astype(int)
        })
        .drop(columns="duration")
    )

duration_pipe1 = Pipeline(steps=[
    ("rbf", RBFPercentileSimilarity()),
    ("scaler", PowerTransformer())
])

duration_pipe2 = Pipeline(steps=[
    ("cat", FunctionTransformer(func=duration_category)),
    ("encoder", OrdinalEncoder(categories=[["short", "medium", "long"]]))
])

duration_union = FeatureUnion(transformer_list=[
    ("part1", duration_pipe1),
    ("part2", duration_pipe2),
    ("part3", FunctionTransformer(func=is_over)),
    ("part4", StandardScaler())
])

duration_transformer = Pipeline(steps=[
    ("outliers", Winsorizer(capping_method="iqr", fold=1.5)),
    ("imputer", SimpleImputer(strategy="median")),
    ("union", duration_union)
])

# total_stops
def is_direct(X):
    return X.assign(is_direct_flight=X.total_stops.eq(0).astype(int))


total_stops_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("", FunctionTransformer(func=is_direct))
])

# additional_info
info_pipe1 = Pipeline(steps=[
    ("group", RareLabelEncoder(tol=0.1, n_categories=2, replace_with="Other")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

def have_info(X):
    return X.assign(additional_info=X.additional_info.ne("No Info").astype(int))

info_union = FeatureUnion(transformer_list=[
("part1", info_pipe1),
("part2", FunctionTransformer(func=have_info))
])

info_transformer = Pipeline(steps=[
("imputer", SimpleImputer(strategy="constant", fill_value="unknown")),
("union", info_union)
])

# column transformer
column_transformer = ColumnTransformer(transformers=[
("air", air_transformer, ["airline"]),
("doj", doj_transformer, ["date_of_journey"]),
("location", location_transformer, ["source", 'destination']),
("time", time_transformer, ["dep_time", "arrival_time"]),
("dur", duration_transformer, ["duration"]),
("stops", total_stops_transformer, ["total_stops"]),
("info", info_transformer, ["additional_info"])
], remainder="passthrough")

# feature selector
estimator = RandomForestRegressor(n_estimators=10, max_depth=3, random_state=42)

selector = SelectBySingleFeaturePerformance(
estimator=estimator,
scoring="r2",
threshold=0.1
) 

# preprocessor
preprocessor = Pipeline(steps=[
("ct", column_transformer),
("selector", selector)
])

In [129]:
preprocessor.fit(
    train.drop(columns="price"),
    train.price.copy()
)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('air',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('grouper',
                                                                   RareLabelEncoder(n_categories=2,
                                                                                    replace_with='Other',
                                                                                    tol=0.1)),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['airline']),
                                                 ('doj',
                                                  Pipeline(steps=[('dt',
                                                                   Datetime...
                                                                                                                                     replace_with='Other',
                                                                                                                                     tol=0.1)),
                                                                                                                   ('encoder',
                                                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                                                  sparse_output=False))])),
                                                                                                  ('part2',
                                                                                                   FunctionTransformer(func=<function have_info at 0x7f7e25f57d90>))]))]),
                                                  ['additional_info'])])),
                ('selector',
                 SelectBySingleFeaturePerformance(estimator=RandomForestRegressor(max_depth=3,
                                                                                  n_estimators=10,
                                                                                  random_state=42),
                                                  scoring='r2',
                                                  threshold=0.1))])

In [130]:
preprocessor.transform(train.drop(columns="price"))

,air__airline_Indigo,air__airline_Jet Airways,air__airline_Other,doj__date_of_journey_week,doj__date_of_journey_day_of_year,location__source,dur__duration_cat,dur__duration_over_1000,dur__duration,stops__total_stops,stops__is_direct_flight
0,0.0,0.0,1.0,0.647059,0.635593,0.156480,1.0,0,-0.567241,1.0,0
1,0.0,1.0,0.0,0.647059,0.661017,0.934312,2.0,1,1.644943,2.0,0
2,1.0,0.0,0.0,0.823529,0.822034,0.934312,1.0,0,-0.619415,1.0,0
3,0.0,1.0,0.0,0.529412,0.516949,0.156480,2.0,1,1.018853,1.0,0
4,0.0,1.0,0.0,0.294118,0.305085,-0.869918,1.0,0,-0.880286,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
635,1.0,0.0,0.0,0.000000,0.016949,0.934312,1.0,0,-0.629850,1.0,0
636,1.0,0.0,0.0,1.000000,1.000000,0.156480,0.0,0,-0.942895,0.0,1
637,0.0,1.0,0.0,0.647059,0.661017,0.156480,2.0,0,0.413633,1.0,0
638,0.0,0.0,1.0,0.411765,0.381356,-0.869918,0.0,0,-0.932460,0.0,1


## 4. Preprocess Data and Upload to Bucket

In [152]:
BUCKET_NAME = "sagemakerflights-bucket"

DATA_PREFIX = "data"

In [153]:
def get_file_name(name):
    return f"{name}-pre.csv"

In [154]:
def export_data(data,name,pre):
     #split data into X and y subsets
     X = data.drop(columns="price")
     y = data.price.copy()
        
     # transformation
     X_pre = pre.transform(X)   
      
     file_name = get_file_name(name)
     (
         y
         .to_frame()
         .join(X_pre)
         .to_csv(file_name,index=False, header=False)
     )   
        

In [155]:
def upload_to_bucket(name):
    file_name = get_file_name(name);
    
    (
        boto3
        .Session()
        .resource("s3")
        .Bucket(BUCKET_NAME)
        .Object(os.path.join(DATA_PREFIX,f"{name}/{name}.csv")) 
        .upload_file(file_name)
    )

In [156]:
def export_and_upload_bucket(data, name, pre):
    export_data(data, name, pre)
    upload_to_bucket(name)

In [157]:
export_and_upload_bucket(train, "train", preprocessor)

In [158]:
export_and_upload_bucket(val, "val", preprocessor)

In [159]:
export_and_upload_bucket(test, "test", preprocessor)

## 5. Model and Hyperparameter Tuning Set-up

In [160]:
session = sagemaker.Session()
region_name = session.boto_region_name

In [161]:
output_path = f"s3://{BUCKET_NAME}/model/output"

In [162]:
model = Estimator(
    image_uri=sagemaker.image_uris.retrieve("xgboost", region_name, "1.2-1"),
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m4.xlarge",
    volume_size=5,
    output_path=output_path,
    use_spot_instances=True,
    max_run=300,
    max_wait=600,
    sagemaker_session=session
)

In [163]:
model.set_hyperparameters(
    objective="reg:linear",
    num_round=10,
    eta=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    alpha=0.1
)

In [164]:
hyperparameter_ranges = {
    "eta": ContinuousParameter(0.05, 0.2),
    "alpha": ContinuousParameter(0, 1),
    "max_depth": IntegerParameter(3, 5)
}

In [165]:
tuner = HyperparameterTuner(
    estimator=model,
    objective_metric_name="validation:rmse",
    hyperparameter_ranges=hyperparameter_ranges,
    strategy="Bayesian",
    objective_type="Minimize"
)

## 6. Data Channels

In [166]:
def get_data_channel(name):
    bucket_path = f"s3://{BUCKET_NAME}/{DATA_PREFIX}/{name}"
    return TrainingInput(bucket_path, content_type="csv")

In [167]:
train_data_channel = get_data_channel("train")
train_data_channel

In [168]:
val_data_channel = get_data_channel("val")

In [169]:
data_channels = {
    "train": train_data_channel,
    "validation": val_data_channel
}

## 7. Train and Tune the Model

In [170]:
tuner.fit(data_channels)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


............................................!


## 8. Model Evaluation

In [188]:
with open("xgboost-model", "rb") as f:
    best_model = pickle.load(f)
    
best_model

In [189]:
pd.read_csv("train-pre.csv").iloc[:,1:]

,0.0,0.0.1,1.0,0.6470588235294118,0.6355932203389831,0.1564797502733027,1.0.1,0,-0.567241011191817,1.0.2,0.1
0,0.0,1.0,0.0,0.647059,0.661017,0.934312,2.0,1,1.644943,2.0,0
1,1.0,0.0,0.0,0.823529,0.822034,0.934312,1.0,0,-0.619415,1.0,0
2,0.0,1.0,0.0,0.529412,0.516949,0.156480,2.0,1,1.018853,1.0,0
3,0.0,1.0,0.0,0.294118,0.305085,-0.869918,1.0,0,-0.880286,0.0,1
4,0.0,0.0,0.0,1.000000,0.974576,0.934312,1.0,0,-0.452458,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...
634,1.0,0.0,0.0,0.000000,0.016949,0.934312,1.0,0,-0.629850,1.0,0
635,1.0,0.0,0.0,1.000000,1.000000,0.156480,0.0,0,-0.942895,0.0,1
636,0.0,1.0,0.0,0.647059,0.661017,0.156480,2.0,0,0.413633,1.0,0
637,0.0,0.0,1.0,0.411765,0.381356,-0.869918,0.0,0,-0.932460,0.0,1


In [190]:
def evaluate_model(name):
    file_name = get_file_name(name)
    data = pd.read_csv(file_name)
    
    X = xgb.DMatrix(data.iloc[:, 1:])
    y = data.iloc[:, 0].copy()
    
    pred = best_model.predict(X)
    
    return r2_score(y, pred)

In [191]:
evaluate_model("train")

0.1311345100402832

In [192]:
evaluate_model("val")

0.0396045446395874

In [193]:
evaluate_model("test")

-0.03758430480957031